# Geometric Distortion Correction

- Input : The original image
- Output : Its rectified image (w/o geometric distortion)
- Given : Its camera matrix and distortion coefficient
- Solutions for the polynomial distortion model
  - OpenCV `cv::undistort()` and `cv::undistortPoints()` (cf. included in imgproc module) <br>
  　　　　　　　　　　　　　↔ `cv::projectPoints()` (cf. included in calib3d module)
  - Camera Distortion Correction : Theory and Practice, http://darkpgmr.tistory.com/31

In [1]:
import cv2
import numpy as np
from copy import deepcopy
%matplotlib inline

In [2]:
## Camera Intrinsic Parameter
fx, fy = 432.7390364738057, 431.2395555913084
cx, cy = 476.0614994349778, 288.7602152621297
K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0, 0, 1]], dtype=np.float32)
## Distorion Coefficient
dist_coeff = np.array([-0.2852754904152874, 0.1016466459919075, -0.0004420196146339175, 0.0001149909868437517, -0.01803978785585194])

In [4]:
# Load and display a .mp4 video.
from IPython.display import HTML
from base64 import b64encode
input_name = "chessboard.mp4"
input = open(input_name,'rb').read()
input_url = "data:video/mp4;base64," + b64encode(input).decode()

In [6]:
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % input_url)

Output hidden; open in https://colab.research.google.com to view.

In [7]:
cap = cv2.VideoCapture(input_name)
if cap.isOpened() == False: raise Exception("No Video")

map1 = np.array([])
map2 = np.array([])
undistorted_images = []
while True:
    ret, image = cap.read()
    if not ret: break

    ## Distortion Correction
    if not(len(map1) & len(map2)):
        map1, map2 = cv2.initUndistortRectifyMap(K, dist_coeff, None, None, (image.shape[1], image.shape[0]), cv2.CV_32FC1)
    undistorted_image = cv2.remap(image, map1, map2, interpolation=cv2.INTER_LINEAR)
    undistorted_images.append(undistorted_image)

height, width, layers = undistorted_images[0].shape
size = (width,height)
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 24, size)
for img in undistorted_images:
  out.write(img)
out.release()

In [8]:
output_name = "output.mp4"
output = open(output_name,'rb').read()
output_url = "data:video/mp4;base64," + b64encode(output).decode()

In [9]:
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % output_url)

Output hidden; open in https://colab.research.google.com to view.